# Env Set Up #

In [1]:
# Import libraries
import os
import cv2
import time
import random
import gc as G
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from numba import cuda
from tensorflow import keras
from tensorflow.keras import backend as K, mixed_precision
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from concurrent.futures import ThreadPoolExecutor
import keras_tuner as kt
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras_tuner import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau as RLOP, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier as KC
from sklearn.model_selection import RandomizedSearchCV as RSCV
from utils import view_train_images, plot_training_results, fast_import2 as FI2, get_images
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Flatten


In [2]:
# limit usage of GPU memory
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Prevents TensorFlow from allocating all memory at once
            print(f"Enabled memory growth for: {gpu}")

        # Set GPU device
        device = "/GPU:0"
    except RuntimeError as e:
        print(e)
else:
    device = "/CPU:0"
    print("No GPU detected, using CPU.")

print(f"Using device: {device}")

# set gpu to use max memory
if gpus:
    try:
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=3072)]  # Set in MB (e.g., 10GB)
            )
        print("Set TensorFlow to use max GPU memory.")
    except RuntimeError as e:
        print(e)


Enabled memory growth for: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Using device: /GPU:0
Set TensorFlow to use max GPU memory.


# 1) Data Import #

In [3]:
# paths
train_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/train"
val_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/val"
test_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/test"

# path to save tuner results
kt_path = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Tuning_Res/KT"

# save best model
model_path = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Tuning_Res/KT/Model"


In [4]:
# data import
train_images, train_labels, val_images, val_labels = get_images(train_dir, train_dir, 25000)

Function processed 25000 images in 25 seconds.

Function processed 3750 images in 10 seconds.



In [8]:
# prepare training data
X_train = np.array(train_images)
X_train = np.repeat(X_train, 3, -1)

Y_train = [str(s) for s in train_labels]
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_hot = to_categorical(Y_train, num_classes=1000)


In [10]:
# prepare testing data
X_val = np.array(val_images)
X_val = np.repeat(X_val, 3, -1)

Y_val = [str(s) for s in val_labels]
label_encoder = LabelEncoder()
Y_val = label_encoder.fit_transform(Y_val)
Y_val_hot = to_categorical(Y_val, num_classes=1000)


In [11]:
# check data shape
print(Y_train.shape)
print(Y_val.shape)

print(len(np.unique(Y_train)))
print(len(np.unique(Y_val)))

(25000,)
(3750,)
1000
973


# 2) Model Optimization #

## a) Keras Classifier ##

In [ ]:
# parameters for tuning
epx = 50
batch_size = 8
RSCV_opts = ['adam', 'rmsprop']
krnl = (3, 3)
in_shape = (224, 224, 3)
RSCV_activation = ['relu', 'tanh', 'sigmoid']
RSCV_node1 = [2, 4, 8, 16, 32]
RSCV_node2 = [2, 4, 8, 16, 32]
RSCV_drop = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
RSCV_bacth = [2, 4, 6, 8, 16, 32, 64, 128]

params_grid = dict(node1 = RSCV_node1,
               node2 = RSCV_node2,
               activations = RSCV_activation,
               optimizers = RSCV_opts,
               batch_size = RSCV_bacth,
               drops = RSCV_drop)

In [ ]:
# Keras Classifier needs a model builder function to instantiate a new model for each new grid point
def build_CNN(optimizers, node1, activations, node2, drops):
    CNN = Sequential([
        # first layer
        Conv2D(node1, kernel_size=krnl, activation=activations, input_shape=in_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # 2nd layer
        Conv2D(node2, kernel_size=krnl, activation=activations),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # output layer
        GlobalAveragePooling2D(),
        Dense(drops),
        Dense(1000, 'softmax')
    ])
    CNN.compile(optimizers,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    # return built model
    return CNN

In [ ]:
# build Keras Classifier
RS_cnn = KC(build_fn=build_CNN,
            verbose=1,
            epochs=5,
            batch_size=batch_size)

In [ ]:
# establish search space and search
grid_search = RSCV(RS_cnn,
                   param_distributions=params_grid,
                   cv=5,
                   scoring='accuracy',
                   verbose=3)

# execute search with categorical Y, not one-hotted
grid_search.fit(X_train, Y_train)

## b) Keras Tuner ##

In [ ]:
# variables for tuning
activations = ['relu', 'leaky_relu', 'swish', 'elu']
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_path + '/ImgNet_Model.h5', monitor='val_loss', save_best_only=True, save_freq='epoch')

In [ ]:
# base model for kt
mixed_precision.set_global_policy('mixed_float16')
def build_KT_CNN(hp):
    layers = [
        # input layer
        Conv2D(hp.Int('filters', min_value=32, max_value=128, step=32),
            input_shape=(128, 128, 3),
            kernel_size=(3, 3),
            activation=hp.Choice('activation', activations)),
            Dropout(hp.Float('drop_rate', 0.1, 0.6))
    ]
    
    # hidden layer(s)
    for l in range(hp.Int('num_layers', 1, 5)):
        layers.append(Conv2D(hp.Int('filters', min_value=32, max_value=128, step=32),
                            kernel_size=(3, 3),
                            activation=hp.Choice('activation', activations)))
    
    # output layer
    layers.append(GlobalAveragePooling2D())
    layers.append(Dropout(hp.Float('drop_rate', 0.1, 0.6)))
    layers.append(Dense(1000, activation=hp.Choice('activation', activations)))
    
    CNN = Sequential(layers)
    lr = hp.Float('learning_rate', min_value=1e-3, max_value=1e-2, sampling='LOG', default=1e-3)
    CNN.compile(optimizer=Adam(learning_rate=lr),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    # return built model
    return CNN

build_KT_CNN(kt.HyperParameters())


In [ ]:
# eastablish search space and search
tuner = kt.RandomSearch(
    hypermodel=build_KT_CNN,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory=kt_path,
    project_name='ImgNetOptimization'
)

# search
tuner.search(X_train, Y_train,
             batch_size=8,
             epochs=10,
             validation_data=(X_test, Y_test),
             callbacks=[early_stop, checkpoint])
tuner.results_summary()

In [ ]:
# view optimized model
model = tuner.get_best_models()[0]
model.summary()

In [ ]:
K.clear_session()
G.collect()

## c) Bayesian Optimizer ##

In [19]:
# variables for tuning
activations = ['relu', 'leaky_relu', 'swish', 'elu']
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_path + '/ImgNet_Model.h5', monitor='val_loss', save_best_only=True, save_freq='epoch')

In [22]:
# base model for Bayesian
def best_modHP(hp):
    layers = [
        # input layer
        Conv2D(hp.Int('filters', min_value=8, max_value=128, step=8),
            input_shape=(64, 64, 3),
            kernel_size=(3, 3),
            activation=hp.Choice('activation', activations)),
            Dropout(hp.Float('drop_rate', 0.1, 0.6))
    ]
    
    # hidden layer(s)
    for l in range(hp.Int('num_layers', 1, 5)):
        layers.append(Conv2D(hp.Int('filters', min_value=8, max_value=128, step=8),
                            kernel_size=(3, 3),
                            activation=hp.Choice('activation', activations))
                            )
        layers.append(BatchNormalization())
        layers.append(MaxPooling2D(2, 2))
        
    
    # output layer
    layers.append(GlobalAveragePooling2D())
    layers.append(Dropout(hp.Float('drop_rate', 0.1, 0.6)))
    layers.append(Dense(1000, activation=hp.Choice('activation', activations)))
    
    CNN = Sequential(layers)
    lr = hp.Float('learning_rate', min_value=1e-3, max_value=1e-2, sampling='LOG', default=1e-3)
    CNN.compile(optimizer=Adam(learning_rate=lr),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    # return built model
    return CNN

In [ ]:
# define search space and search
tuner_bay = BayesianOptimization(best_modHP,
                                 objective='val_acc',
                                 max_trials=5,
                                 executions_per_trial=1,
                                 project_name='Bayesian',
                                 overwrite=True)

# search
tuner_bay.search(X_train, Y_train,
                 epochs=20,
                 validation_data=(X_val, Y_val),
                 callbacks=[early_stop, checkpoint])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
96                |96                |filters
leaky_relu        |leaky_relu        |activation
0.36209           |0.36209           |drop_rate
1                 |1                 |num_layers
0.0017833         |0.0017833         |learning_rate

Epoch 1/20
782/782 [==============================] - 32s 39ms/step - loss: 7.2104 - accuracy: 0.0011 - val_loss: 11.2045 - val_accuracy: 5.3333e-04
Epoch 2/20
782/782 [==============================] - 30s 39ms/step - loss: 6.9110 - accuracy: 8.4000e-04 - val_loss: 9.3220 - val_accuracy: 5.3333e-04
Epoch 3/20
782/782 [==============================] - 30s 39ms/step - loss: 6.9153 - accuracy: 0.0014 - val_loss: 9.4378 - val_accuracy: 0.0019
Epoch 4/20
782/782 [==============================] - 30s 39ms/step - loss: 7.0812 - accuracy: 9.6000e-04 - val_loss: 11.9844 - val_accuracy: 8.0000e-04
Epoch 5/20
782/782 [==============================] - 30s 39ms/step - loss: 6

In [17]:
# clear gpu
K.clear_session()
G.collect()

105